## Ищем многочлены Чебышова

Напомним, что алгоритмы SOS-оптимизации умеют решать следующую задачу.

**Вход:** многочлен $p(x_1, \ldots, x_n, c_1, \ldots, c_m)$ линейный относительно $c_1, \ldots, c_m$ и линейная функция $\ell(c_1, \ldots, c_m)$

**Выход:** Максимальное значение $\ell(c_1, \ldots, c_m)$ среди наборов чисел $c_1, \ldots, c_m$, для которых $p(x_1, \ldots, x_n, c_1, \ldots, c_m)$ - SOS относительно $x_1, \ldots, x_n$.

**Пример:** Чему равно наибольшее значение $a$, при котором $x^2 + ax + 4$ является суммой квадратов?

In [1]:
using SumOfSquares
using DynamicPolynomials
using Mosek
using MosekTools
using SemialgebraicSets

In [2]:
opt = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true);
model = SOSModel(opt)

@polyvar z
S = @set -1 <= z && z <= 1 # живем на [-1, 1]

d = 10
@variable(model, a[0:d])
p = sum([a[i] * z^i for i in 0:d])

(a[10])z¹⁰ + (a[9])z⁹ + (a[8])z⁸ + (a[7])z⁷ + (a[6])z⁶ + (a[5])z⁵ + (a[4])z⁴ + (a[3])z³ + (a[2])z² + (a[1])z + (a[0])

In [3]:
@objective(model, Max, a[d]) # функция L, которую оптимизируем
@constraint(model, p <= 1, domain = S)
@constraint(model, p >= -1, domain = S)

(a[10])z¹⁰ + (a[9])z⁹ + (a[8])z⁸ + (a[7])z⁷ + (a[6])z⁶ + (a[5])z⁵ + (a[4])z⁴ + (a[3])z³ + (a[2])z² + (a[1])z + (a[0] + 1) is SOS

In [4]:
optimize!(model)

In [5]:
value(p)

511.9999962549675z¹⁰ + 3.294303089955077e-8z⁹ - 1279.999990250578z⁸ - 6.894012698563223e-8z⁷ + 1119.9999911124535z⁶ + 4.701037338930788e-8z⁵ - 399.9999967084261z⁴ - 1.1565773067861164e-8z³ + 49.99999957877693z² + 7.358655835241071e-10z - 0.9999999910424843

In [6]:
round(value(p), digits = 3)

512.0z¹⁰ - 1280.0z⁸ + 1120.0z⁶ - 400.0z⁴ + 50.0z² - 1.0

## Эксперимент: Наименьшее уклонение на двух отрезках сразу

In [7]:
opt = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true);
model = SOSModel(opt)

@polyvar z
S = @set (z - 2) * (z - 1) * (z + 1) * (z + 2) <= 0 # NEW
# живем [-2, -1] в [1, 2]

d = 4
@variable(model, a[0:d])
p = sum([a[i] * z^i for i in 0:d])

@objective(model, Max, a[d])
@constraint(model, p <= 1, domain = S)
@constraint(model, p >= -1, domain = S)

optimize!(model)

In [8]:
value(p)

0.8888888896752403z⁴ - 4.444444446977403z² + 4.5555555564390575

In [9]:
round(value(p), digits = 5)

0.88889z⁴ - 4.44444z² + 4.55556

Видно, что здесь приятные рациональные числа! Для других значений d всё менее очевидно - надо раскладывать в цепные дроби и подбирать. Если есть интерес - смело пробуйте